# 8 puzzle problem solution by A*

- Student Name: Pablo Muñoz Haro
- Student ID: A01222422
- Course name: TC2011 INTELLIGENT SYSTEMS
- Date: August 12th, 2018
- Assignment Name: A* algorithm using Manhattan heuristic
- Python version: 3.6.4

The 8 puzzle problem is defined as follows: Consider a 3x3 (nine cell) grid. One of the cells is "blank" and the rest contains the integers 1 through 8 without repetitions. Obvisouly the grid has the numbers and the blank ordered in some initial arrangement. It is desired to move the "cells" around until the grid is in another, target arrangement (say with the blank first and then the numbers in increasing order). You can only move the cells adjacent to the blank and only to the blank's position (think of it as you can swap the blank and one of its neighbors positions). The goal is to come up with a move sequence (left, right, up down) that will take you from the initial configuration to the target configuration.

The solution by A* consists of possible moves perceived to be "low cost" at each stage, for this problem, the state of the grid is encoded as a 9-tuple filled with integers. We use a 0 to denote the "blank" and the numbers 1 through 8 represent themselves. For example, the state (1, 3, 5, 7, 0, 8, 2, 4, 6) represents the grid:

```
| 1 | 3 | 5 |
| 7 |   | 8 |
| 2 | 4 | 6 |
```

Tuples are used to store the grid state due to their ability to be hashable, which allows us to use them in sets and test for membership easily. For the queue we use python's standard `deque` which allow for very efficient additions and removals both at the start and at the end of the collection.

This notebook is an alteration of the 0380_Assignment1_BFS_8puzzle notebook. The majority of the work is still being done by the function `eight_puzzle_a_star(initial_state, final_state)`.
Unlike the function for the BFS approach, this time the function mantains a min priority queue that starts with the initial state. Then, while the queue is not empty and the goal hasn't been reached an element is dequeued from the queue, and every possible neighbor is calculated and put back into the queue (a neighbor is a state that would result from doing an UP, DOWN, LEFT or RIGHT, move). `eight_puzzle_a_star` implements an inner function `swap` that just makes it easy to swap two elements in a list. The function also implements the inner function `cost function` which computes an approximation of the cost from transitioning from a given state to the final or goal state.

To use this notebook simply modify the FINAL_STATE and INITIAL_STATE variables tha appear a few cells below.

In [1]:
from collections import namedtuple
import time
import heapq

In [2]:
eight_puzzle_solution = namedtuple('eight_puzzle_solution', [
    'path_to_goal', 'path_cost', 'num_visited_nodes',
    'running_time_secs', 'used_memory'
])

In [3]:
FINAL_STATE = (1, 2, 3, 4, 5, 6, 7, 8, 0)

In [4]:
INITIAL_STATE = (4, 1, 3, 0, 2, 6, 7, 5, 8)

In [5]:
def eight_puzzle_a_star(initial_state, final_state, print_heap=True):
    '''
    Solves the eight puzzle problem through the a*
    search algorithm.
    
    Args:
        initial_state (tuple): A tuple representing the initial
            state of the problem. It is assumed that the tuple is
            of length 9 and contains the numbers 0 through 8 without
            repetitions.
        final_state (tuple): A tuple representing the desired final
            state of the problem. It is assumed that the tuple is
            of length 9 and contains the numbers 0 through 8 without
            repetitions.
          
    Returns:
        eight_puzzle_solution namedtuple: A number of statistics
            about the solution and the effor it took to achieve it.
    '''
    BRANCHING_FACTOR = 9
    UNIT_COST = 72
    
    def swap(container, index_1, index_2):
        '''
        Utility function to swap two elements of a collection.
        Used to calculate the next state after moving the 8-puzzle
        pivot one spot to the left, right, up or down.
        
        Args:
            container (list): A mutable representation of a state
            index_1 (int): One of the indexes to swap
            index_2 (int): The other index to swap
            
        Returns:
            None
            
        Side-Effects:
            Mutates container so that elements at positions
            index_1 and index_2 are swapped
        '''
        container[index_1], container[index_2] = container[index_2], container[index_1]
        
    def cost_function(current_state, final_state):
        '''
        Utility function that calculates the approximated cost
        to get from the current_state to the final_state. It uses
        the manhattan distance to approximate the cost. This is done
        by assigning a cost of 1 for a cell that is different in
        the current_state than in the final_state and a cost of 0
        for cells that are equal. This is calculated for all cells
        and summed over.
        
        Args:
            current_state (tuple): 9-tuple of integers 0 through 8 without repetitions
            final_state (tuple): 9-tuple of integers 0 through 8 without repetitions
            
        Returns
            int: A metric that represents the approximate cost for getting from
                the current_state to the final_state.
        '''
        return sum(1 if current_state[i] is not final_state[i] else 0 for i in range(len(current_state)))
    
    frontier = [(0, initial_state)]
    visited = set()
    start_time = time.time()
    max_queue_size = 1 # Mantain a count of the most number of nodes held in memory at any point
    
    move_sequences = {
        INITIAL_STATE: ''
    }
    
    if print_heap: print('Frontiers through time (heap):')
    
    while len(frontier) is not 0:
        if len(frontier) > max_queue_size:
            max_queue_size = len(frontier)
            
        if print_heap: print(frontier)
        
        cost, node_to_expand = heapq.heappop(frontier)
        visited |= {node_to_expand}
        
        if node_to_expand == FINAL_STATE:
            end_time = time.time()
            elapsed_time = end_time - start_time
            path_to_goal = move_sequences[FINAL_STATE]
            path_cost = len(path_to_goal)
            num_visited_nodes = len(visited)
            used_memory = max_queue_size*UNIT_COST
            
            return eight_puzzle_solution(
                path_to_goal,
                path_cost,
                num_visited_nodes,
                elapsed_time,
                used_memory,
            )
        
        pivot_index = node_to_expand.index(0)
        neighbors = []
        
        can_move_left = pivot_index not in (0, 3, 6)
        can_move_right = pivot_index not in (2, 5, 8)
        can_move_up = pivot_index not in (0, 1, 2)
        can_move_down = pivot_index not in (6, 7, 8)
        
        if can_move_left:
            left_neighbor_listform = list(node_to_expand)
            swap(left_neighbor_listform, pivot_index, pivot_index - 1)
            left_neighbor = tuple(left_neighbor_listform)
            if left_neighbor not in move_sequences:
                move_sequences[left_neighbor] = move_sequences[node_to_expand] + 'L'
            neighbors.append(left_neighbor)
            
        if can_move_right:
            right_neighbor_listform = list(node_to_expand)
            swap(right_neighbor_listform, pivot_index, pivot_index + 1)
            right_neighbor = tuple(right_neighbor_listform)
            if right_neighbor not in move_sequences:
                move_sequences[right_neighbor] = move_sequences[node_to_expand] + 'R'
            neighbors.append(right_neighbor)

        if can_move_up:
            up_neighbor_listform = list(node_to_expand)
            swap(up_neighbor_listform, pivot_index, pivot_index - 3)
            up_neighbor = tuple(up_neighbor_listform)
            if up_neighbor not in move_sequences:
                move_sequences[up_neighbor] = move_sequences[node_to_expand] + 'U'
            neighbors.append(up_neighbor)

        if can_move_down:
            down_neighbor_listform = list(node_to_expand)
            swap(down_neighbor_listform, pivot_index, pivot_index + 3)
            down_neighbor = tuple(down_neighbor_listform)
            if down_neighbor not in move_sequences:
                move_sequences[down_neighbor] = move_sequences[node_to_expand] + 'D'
            neighbors.append(down_neighbor)
            
        for neigh in neighbors:
            if neigh not in frontier and neigh not in visited:
                heapq.heappush(frontier, (cost_function(neigh, final_state), neigh))

In [6]:
solution = eight_puzzle_a_star(INITIAL_STATE, FINAL_STATE)

Frontiers through time (heap):
[(0, (4, 1, 3, 0, 2, 6, 7, 5, 8))]
[(5, (0, 1, 3, 4, 2, 6, 7, 5, 8)), (6, (4, 1, 3, 2, 0, 6, 7, 5, 8)), (7, (4, 1, 3, 7, 2, 6, 0, 5, 8))]
[(4, (1, 0, 3, 4, 2, 6, 7, 5, 8)), (7, (4, 1, 3, 7, 2, 6, 0, 5, 8)), (6, (4, 1, 3, 2, 0, 6, 7, 5, 8))]
[(3, (1, 2, 3, 4, 0, 6, 7, 5, 8)), (5, (1, 3, 0, 4, 2, 6, 7, 5, 8)), (6, (4, 1, 3, 2, 0, 6, 7, 5, 8)), (7, (4, 1, 3, 7, 2, 6, 0, 5, 8))]
[(2, (1, 2, 3, 4, 5, 6, 7, 0, 8)), (4, (1, 2, 3, 4, 6, 0, 7, 5, 8)), (4, (1, 2, 3, 0, 4, 6, 7, 5, 8)), (7, (4, 1, 3, 7, 2, 6, 0, 5, 8)), (5, (1, 3, 0, 4, 2, 6, 7, 5, 8)), (6, (4, 1, 3, 2, 0, 6, 7, 5, 8))]
[(0, (1, 2, 3, 4, 5, 6, 7, 8, 0)), (4, (1, 2, 3, 4, 6, 0, 7, 5, 8)), (3, (1, 2, 3, 4, 5, 6, 0, 7, 8)), (7, (4, 1, 3, 7, 2, 6, 0, 5, 8)), (5, (1, 3, 0, 4, 2, 6, 7, 5, 8)), (6, (4, 1, 3, 2, 0, 6, 7, 5, 8)), (4, (1, 2, 3, 0, 4, 6, 7, 5, 8))]


In [7]:
print('Path to goal: {}'.format(solution.path_to_goal))
print('Cost of path: {}'.format(solution.path_cost))
print('# visisted nodes: {}'.format(solution.num_visited_nodes))
print('Running time: {}'.format(solution.running_time_secs))
print('Memory used: {}'.format(solution.used_memory))

Path to goal: URDDR
Cost of path: 5
# visisted nodes: 6
Running time: 0.00033402442932128906
Memory used: 504


In [8]:
# Solving the same problem as the other BFS notebook to demonstrate A* has better performance.
# Although, it lead to a more complicated solution. For reference, the statistics for BFS
# were the following:
# Path to goal: LURDRDLLURRDLLURRULLDRRULL
# Cost of path: 26
# # visisted nodes: 164919
# Running time: 219.45377707481384
# Memory used: 1798776
INITIAL_STATE = (7, 2, 4, 5, 0, 6, 8, 3, 1)
FINAL_STATE = (0, 1, 2, 3, 4, 5, 6, 7, 8)
solution = eight_puzzle_a_star(INITIAL_STATE, FINAL_STATE, print_heap=False)
print('Path to goal: {}'.format(solution.path_to_goal))
print('Cost of path: {}'.format(solution.path_cost))
print('# visisted nodes: {}'.format(solution.num_visited_nodes))
print('Running time: {}'.format(solution.running_time_secs))
print('Memory used: {}'.format(solution.used_memory))

Path to goal: ULDRULDRURDLDLURURDDLUURDLDLUURRDLLURDRULDLDRUULDRDLUURDRULLDRRULDLURDLURRDLLURDRULDLDRUULDRDLUURDLDRUULDRDLUU
Cost of path: 110
# visisted nodes: 489
Running time: 0.012885093688964844
Memory used: 24984
